In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
from hard_data import generate_data_head_shoulder, generate_sample_df_with_pattern
from analysis import detect_and_rename

In [4]:
# Read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Get the data for these tickers from yahoo finance
# yf_top500 = yf.download(tickers.Symbol.to_list(), interval='60m', period='2y', auto_adjust=True)['Close']

In [8]:
# Plot the chart. Saves it using the id
def plot_simple_df(candle_data: pd.DataFrame, stock_type: str, ticker: str,  count: int, pad: int = 2):
    if pad < 0:
        pad = 0
    fname = "images/hs/" + stock_type + ticker + '_' + str(count) + ".png"
    
    candle_data.index = pd.DatetimeIndex(candle_data['Datetime'])
    candle_data.drop(columns=['Datetime'], inplace=True)

    mpf.plot(candle_data, type='candle', axisoff=True, style = 'classic', savefig=dict(fname=fname, dpi=60))  

In [ ]:
for ticker in tickers.Symbol.to_list()[0:2]:
    try:
        data = pd.read_csv('data/' + ticker + '_data.csv', index_col=None, header=0)
        # Place the datetime in a temp dataset
        temp = data['Datetime']
        data.rename(columns={"open": "Open", "close": "Close", "high": "High",
                             "low": "Low", 'volume': 'Volume'}, inplace=True)
        # Apply numeric and log to the data
        data = data.apply(pd.to_numeric, errors='coerce')
        data = np.log(data)
        # Add the datetime we saved earlier back in
        data['Datetime'] = temp
        data.drop(columns=['adj_close', 'adj_high', 'adj_low'], inplace=True)
        
        data = detect_and_rename(data, 'wavelet', 3)
        data1 = data[data['head_shoulder_pattern_wavelet'].notnull()]
        
        loc = 0
        for i in data1.index:
            if i + 15 < len(data) and loc < i and i > 15:
                loc += 40
                plot_simple_df(data.loc[i - 15 : i+15], 'hs_', ticker, loc, 2)   
    except Exception as e:
        print(f"Failed on ticker {ticker} with exception {e}")
        continue

Failed on ticker BRK.B with exception [Errno 2] No such file or directory: 'data/BRK.B_data.csv'
Failed on ticker BF.B with exception [Errno 2] No such file or directory: 'data/BF.B_data.csv'
Failed on ticker COR with exception [Errno 2] No such file or directory: 'data/COR_data.csv'
Failed on ticker ELV with exception [Errno 2] No such file or directory: 'data/ELV_data.csv'
Failed on ticker EG with exception [Errno 2] No such file or directory: 'data/EG_data.csv'
Failed on ticker GEN with exception [Errno 2] No such file or directory: 'data/GEN_data.csv'
Failed on ticker RVTY with exception [Errno 2] No such file or directory: 'data/RVTY_data.csv'


RuntimeError: the tracemalloc module must be tracing memory allocations to take a snapshot